# Datensatz mit generierten Daten erstellen

In [ ]:
#import necessary libraries
import pandas as pd
import numpy as np
import panel as pn
import hvplot.xarray
import hvplot.pandas
import datetime as dt
from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns
from itertools import cycle
import mysql.connector

pn.extension('tabulator')
pn.extension()
from sqlalchemy import create_engine

In [ ]:
#read dataset
data = pd.read_csv('example_data_set.csv')

#make data interactive
data_int = data.interactive()

In [ ]:
# add subject column
subject_list = cycle(['Deutsch', 'Englisch', 'Mathematik', 'Biologie', 'Musik', 'Kunst', 'Sport'])    
data['subject'] = [next(subject_list) for subject in range(len(data))]

# add class column
class_list = cycle(['Klasse 1a', 'Klasse 1d', 'Klasse 3b', 'Klasse 4c', 'Klasse 5a'])
data['class'] = [next(class_list) for klasse in range(len(data))]

# add exercise column
exercise_list = cycle(['Aufgabe 1', 'Aufgabe 2', 'Aufgabe 3', 'Aufgabe 4', 'Aufgabe 5', 'Aufgabe 6'])
data['exercise'] = [next(exercise_list) for exercise in range(len(data))]

# add 'Allgemein' column
data['Allgemein'] = 'Allgemein'

# add datetime column and convert to datetime object
data['datetime'] = data['date'].astype(str) + ' ' + data['time']
data['datetime'] = data['datetime'].str.replace(',', '')
#data['datetime'] = pd.to_datetime(data['datetime'], '%b %d %Y %H:%M:%S')
data['datetime'] = pd.to_datetime(data['datetime'])

data['Alle Fächer'] = 'Alle Fächer'

In [ ]:
# all the lists
subject_list = data['subject'].unique().tolist()
class_list = data['class'].unique().tolist()
exercise_list = data['exercise'].unique().tolist()

#name_list = data['name'].values.tolist()
name_list = data['name'].unique().tolist()
name_list.sort()

default_exercise = ['Allgemein']
class_list_with_none = ['Keine Auswahl'] + class_list
name_list_with_none = ['Keine Auswahl'] + name_list

In [ ]:
data = data.rename(columns = {'name': 'Name',
                              'duration': 'Bearbeitungsdauer',
                              'number of tries': 'Anzahl Versuche',
                              'subject': 'Fach',
                              'exercise': 'Aufgabe',
                              'class': 'Klasse',
                              'score': 'Punktzahl',
                              'datetime': 'Startzeit'
                             })

In [ ]:
data = data.replace(26, np.nan)

In [ ]:
data['Alle Fächer'] = 'Alle Fächer'
data['Klasse'] = data['Klasse'].apply({'Klasse 1d': 'Klasse 1a', 'Klasse 4c': 'Klasse 3b', 'Klasse 5a': 'Klasse 8.2', 'Klasse 1a': 'Klasse 1a', 'Klasse 3b': 'Klasse 3b'}.get)
data = data.drop(columns = ['date', 'time', 'passed'])
data = data.rename(columns = {'Bearbeitungsdauer': 'Quiz Bearbeitungsdauer', 'Startzeit': 'Quiz Startzeit'})

# https://stackoverflow.com/questions/53543061/convert-string-to-timedelta-in-pandas
temp = data['Quiz Bearbeitungsdauer'].str.split(':', expand = True).astype(int)
new_column = pd.to_timedelta(temp[0], unit = 'h') + pd.to_timedelta(temp[1], unit = 'm') + pd.to_timedelta(temp[2], unit = 's')
data['Quiz Bearbeitungsdauer'] = new_column
data['Quiz Abgabezeit'] = data['Quiz Startzeit'] + data['Quiz Bearbeitungsdauer']

data['Maximale Punktzahl'] = 100
data['Maximale Note in %'] = 100
data['Beste Punktzahl in %'] = data['Punktzahl']

In [ ]:
for e in exercise_list:
    for s in subject_list:
        data.loc[((data['Fach'] == str(s)) & (data['Aufgabe'] == str(e)), 'Aufgabe')] = str(s) + ' ' + str(e)

In [ ]:
data.to_csv('extension_for_moodle_data.csv')